In [5]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
WEB_URL="https://eumostwanted.eu"

In [7]:
driver = webdriver.Firefox()
driver.get(WEB_URL)

(driver.page_source).encode('utf-8')

b'<html lang="en" dir="ltr" class="js"><head>\n    <meta charset="utf-8">\n<script type="text/javascript" defer="" async="" src="/sites/default/files/matomo/matomo.js?sqqoqm"></script><script>var _paq = _paq || [];(function(){var u=(("https:" == document.location.protocol) ? "https://eumostwanted.eu/piwik/" : "https://eumostwanted.eu/piwik/");_paq.push(["setSiteId", "1"]);_paq.push(["setTrackerUrl", u+"matomo.php"]);_paq.push(["setDoNotTrack", 1]);_paq.push(["disableCookies"]);if (!window.matomo_search_results_active) {_paq.push(["trackPageView"]);}_paq.push(["setIgnoreClasses", ["no-tracking","colorbox"]]);_paq.push(["enableLinkTracking"]);var d=document,g=d.createElement("script"),s=d.getElementsByTagName("script")[0];g.type="text/javascript";g.defer=true;g.async=true;g.src="/sites/default/files/matomo/matomo.js?sqqoqm";s.parentNode.insertBefore(g,s);})();</script>\n<meta name="Generator" content="Drupal 10 (https://www.drupal.org)">\n<meta name="MobileOptimized" content="width">\n<m

In [ ]:
wanted_items = list()
result = list()

cookie_deny = driver.find_element(By.ID, "sort_close")
cookie_deny.click()

time.sleep(0.5)

aux_wanted_items = driver.find_elements(by=By.CLASS_NAME, value="wantedItem")

for item in aux_wanted_items:
    aux = item.find_element(by=By.CLASS_NAME, value="content").text
    if (aux.startswith("Arrest")):
        continue

    wanted_items.append(aux)

for item in wanted_items[:]:
    try:
        item = item.replace("\n", "")
        search_bar = driver.find_element(By.NAME, "keys")

        search_bar.clear()
        search_bar.send_keys(item)
        search_bar.submit()
        
        time.sleep(0.7)

        driver.get(driver.execute_script("return window.location.href;"))
        
        element = driver.find_element(By.TAG_NAME, "h3")
        element.click()

        time.sleep(0.1)

        driver.get(driver.execute_script("return window.location.href;"))
        
        import requests, base64
        
        time.sleep(0.7)

        driver.execute_script("scroll(0, 400);")

        time.sleep(0.7)
        
        driver.save_screenshot(f"Screenshots/{ item.replace(",", "").replace(" ", "_") }.png")

        image = driver.find_element(By.CLASS_NAME, "wanted_top_left").find_element(By.TAG_NAME, "img").get_attribute("src")
        base64_string = base64.b64encode(requests.get(image).content).decode("utf-8")

        wanted = driver.find_element(By.CLASS_NAME, "field--node-field-enfast-country").text.replace("by", "").strip()
        bandera = driver.find_element(By.CLASS_NAME, "flag").get_attribute("outerHTML")

        datas = driver.find_elements(By.CLASS_NAME, "clearfix")

        res = dict()
        res["Nombre"] = item
        res["Buscado_en"] = wanted
        res["Bandera"] = bandera

        for data in datas:
            
            clave = data.find_element(By.CLASS_NAME, "field-label").text
            info = data.find_element(By.CLASS_NAME, "field__item").text

            res[clave] = info
            
        res["Imagen"] = base64_string

        result.append(res)
    
    except(Exception) as error:
        print("Error: ", item, error)
    time.sleep(0.2)

driver.quit()

import json

result = json.dumps(result, indent=4)
with open("result.json", "w") as file:
    file.write(result)